# Laboratorium 6 - rekomendacje grupowe

## Przygotowanie

 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab6`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas matplotlib`

## Część 1. - przygotowanie danych

In [1]:
# importujemy wszystkie potrzebne pakiety

import math
import numpy as np
import pandas

from random import choice, sample
from statistics import mean, stdev

from reco_utils import *

In [3]:
# wczytujemy oceny uytkownikow i obliczamy (za pomocą collaborative filtering) wszystkie przewidywane oceny filmow

raw_ratings = pandas.read_csv('ml-latest-small/ratings.csv').drop(columns=['timestamp'])
movies = list(raw_ratings['movieId'].unique())
users = list(raw_ratings['userId'].unique())
ratings = get_predicted_ratings(raw_ratings)
ratings

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
1,10,9,8,9,10,8,8,8,8,9,...,8,5,7,7,8,8,10,9,9,7
2,6,5,5,5,6,4,4,5,4,5,...,5,3,4,4,4,5,6,5,5,4
3,4,4,3,4,4,3,3,3,3,4,...,3,2,3,3,3,3,4,4,4,3
4,9,7,7,7,8,6,7,7,6,7,...,6,4,6,5,6,6,8,8,7,6
5,6,5,5,5,6,5,4,5,5,5,...,5,3,4,4,4,5,6,5,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,9,8,7,7,9,7,7,7,7,8,...,7,4,6,7,6,7,8,8,8,6
607,9,8,7,8,9,7,7,7,6,8,...,7,5,6,6,6,7,8,8,8,6
608,8,7,6,7,8,6,6,6,6,7,...,6,4,5,5,6,6,7,7,7,5
609,6,5,4,5,5,4,5,4,4,5,...,4,3,4,4,4,4,5,5,5,4


In [4]:
# definiujemy testowe grupy uzytkownikow, dla ktorych bedziemy generowac rekomendacje

groups_no = 50
group_size = 5
groups = [sample(users, group_size) for i in range(groups_no)]
groups

[[466, 123, 302, 3, 408],
 [122, 332, 497, 605, 50],
 [237, 366, 275, 428, 34],
 [544, 596, 463, 66, 431],
 [53, 286, 477, 483, 549],
 [431, 348, 464, 9, 524],
 [605, 49, 519, 328, 218],
 [364, 117, 213, 56, 105],
 [513, 137, 86, 38, 458],
 [6, 539, 574, 158, 390],
 [102, 97, 533, 31, 272],
 [498, 517, 59, 12, 439],
 [227, 432, 362, 429, 336],
 [388, 163, 332, 570, 317],
 [438, 312, 351, 571, 546],
 [312, 476, 482, 118, 579],
 [517, 336, 107, 57, 359],
 [501, 80, 379, 134, 149],
 [199, 369, 332, 46, 494],
 [326, 141, 184, 391, 482],
 [573, 356, 299, 482, 277],
 [578, 365, 274, 397, 410],
 [286, 178, 103, 199, 81],
 [284, 20, 323, 430, 136],
 [554, 478, 81, 201, 477],
 [368, 151, 285, 100, 158],
 [145, 182, 464, 135, 95],
 [299, 496, 460, 17, 102],
 [196, 350, 394, 4, 474],
 [311, 474, 401, 296, 254],
 [589, 488, 65, 486, 64],
 [8, 45, 425, 560, 196],
 [512, 441, 188, 173, 132],
 [184, 104, 273, 528, 217],
 [401, 218, 595, 261, 10],
 [439, 48, 65, 147, 8],
 [548, 208, 205, 397, 490],
 [

## Część 2. - algorytmy proste

In [5]:
# zdefiniujmy interfejs dla wszystkich algorytmow rekomendacyjnych

class Recommender:
    def recommend(self, movies, ratings, group, size):
        pass


# jako pierwszy zaimplementujemy algorytm losowy - dla porownania
    
class RandomRecommender(Recommender):
    def __init__(self):
        self.name = 'random'
        
    def recommend(self, movies, ratings, group, size):
        return sample(movies, size)

In [6]:
# algorytm rekomendujacy filmy o najwyzszej sredniej ocen

class AverageRecommender(Recommender):
    def __init__(self):
        self.name = 'average'
    
    def recommend(self, movies, ratings, group, size):
        recommendations = movies
        recommendations.sort(key=lambda m: np.sum([ratings[user][m] for user in group]))
        return recommendations[:size]

In [15]:
# algorytm rekomendujacy filmy o najwyzszej sredniej ocen,
#   ale rownoczesnie wykluczajacy te filmy, ktore otrzymaly choc jedna ocene ponizej thresholdu

class AverageWithoutMiseryRecommender(Recommender):
    def __init__(self, score_threshold):
        self.name = 'average_without_misery'
        self.score_threshold = score_threshold
        
    def recommend(self, movies, ratings, group, size):
        recommendations = [movie for movie in movies if min([ratings[user][m1] for user in group] > self.score_threshold)]
        recommendations.sort(key=lambda m: np.average([ratings[user][m] for user in group]))
        return recommendations[:size]

In [10]:
# algorytm uwzgledniajacy preferencje tylko jednego uzytkownika

class DictatorshipRecommender(Recommender):
    def __init__(self, dictator_id):
        self.name = 'dictatorship'
        self.dictator_id = dictator_id
        
    def recommend(self, movies, ratings, group, size):
        recommendations = movies
        recommendations.sort(key=lambda m: ratings[self.dictator_id][m])
        return recommendations[:size]

In [14]:
# algorytm, ktory w kazdej turze uwzglednia preferencje tylko jednego, kolejnego uzytkownika

class FairnessRecommender(Recommender):
    def __init__(self):
        self.name = 'fairness'
    
    def recommend(self, movies, ratings, group, size):
        recommendations = []
        for turn in range(size):
            recommendations.append(max(list(ratings[group[turn]])))
        return recommendations

In [17]:
# algorytm zachlanny, aproksymujacy metode Proportional Approval Voting
#   w kazdej iteracji wybieramy ten film, ktory najbardziej zwieksza zadowolenie zgodnie z punktacja PAV

class PAVRecommender(Recommender):
    def __init__(self, threshold):
        self.threshold = threshold
        self.name = 'PAV'
        
    def recommend(self, movies, ratings, group, size):
        satisfied_counter = np.ones(len(group))
        recommendations = []
        movies = [movie for movie in movies if min([ratings[user][m1] for user in group] > self.score_threshold)]
        for turn in range(size):
            scores = [np.sum([1/satisfied_counter[idx] if ratings[user][movie] else 0 for idx, user in enumerate(group)]) for 
                     movie in movies]
            best_movie_idx = max(range(scores), key=lambda i: scores[i])
            recommendations.append(movies[best_movie_idx])
            for idx, user in enumerate(group):
                satisfied_counter[idx] += 1 if ratings[user][recommendations[-1]] > self.threshold else 0
            

## Część 3. - funkcje celu

In [18]:
# dwie funkcje pomocnicze:
#  - znajdujaca ulubione filmy danego uzytkownika
#  - obliczajaca sume ocen wystawionych przez uzytkownika wszystkim filmom w rekomendacji

def top_n_movies_for_user(ratings, movies, user_id, n):
    movies.sort(key=lambda m: ratings[user_id][m])
    return movies[:n]

def total_score(recommendation, user_id, ratings):
    return np.sum([ratings[user_id][m] for m in recommendation])

In [ ]:
# funkcja obliczajaca zadowolenie pojedynczego uzytkownika
#  - iloraz zadowolenia z wygenerowanej rekomendacji oraz zadowolenia z hipotetycznej rekomendacji idealnej
def overall_user_satisfaction(recommendation, user_id, movies, ratings):
    

# funkcja celu - srednia z zadowolenia wszystkich uzytkownikow w grupie
def overall_group_satisfaction(recommendation, group, movies, ratings):
    return 1.0 * sum([overall_user_satisfaction(recommendation, user_id, movies, ratings) for user_id in group]) / len(group)

# funkcja celu - roznica miedzy maksymalnym i minimalnym zadowolenie w grupie
def group_dissatisfaction(recommendation, group, movies, ratings):
    satisfaction_scores = [overall_user_satisfaction(recommendation, user_id, movies, ratings) for user_id in group]
    return max(satisfaction_scores) - min(satisfaction_scores)

## Część 4. - Sequential Hybrid Aggregation

In [ ]:
# algorytm balansujacy pomiedzy wyborem elementow o najwyzszej sredniej ocen
#   i o najwyzszej minimalnej ocenie
#   wyliczajacy w kazdej iteracji parametr alfa - jak na wykladzie
class SequentialHybridAggregationRecommender(Recommender):
     def __init__(self, dictator_id):
        self.name = 'sequential_hybrid_aggregation'
    
    def recommend(self, movies, ratings, group, size):
        pass

## Część 5. - porównanie algorytmów

In [ ]:
recommenders = [
    RandomRecommender(),
    AverageRecommender(),
    AverageWithoutMiseryRecommender(2),
    DictatorshipRecommender(1),
    FairnessRecommender(),
    PAVRecommender(),
    SequentialHybridAggregationRecommender()
]

recommendation_size = 10

# dla kazdego algorytmu:
#  - wygenerujmy jedna rekomendacje dla kazdej grupy
#  - obliczmy wartosci funkcji celu dla kazdej rekomendacji
#  - obliczmy srednia i odchylenie standardowe wartosci funkcji celu
#  - wypiszmy wyniki na konsole

for recommender in recommenders:
    # ...